In [29]:
%matplotlib inline

import glob
import numpy as np
import sklearn.linear_model
import sys
import os
import json

In [30]:
def load_vball_data():
    games = []
    for vball_json_filename in glob.glob('data/json/*.json'):
        with open(vball_json_filename, 'r') as vball_json:
            games.append(json.load(vball_json))
    return games

GAMES = load_vball_data()

In [31]:
GAMES[0]

{u'games': [{u'left': u'team_a',
   u'left_score': 26,
   u'left_timeouts': [[21, 23]],
   u'right': u'team_b',
   u'right_score': 24,
   u'right_timeouts': [[16, 17]],
   u'scores': [[0, 1],
    [0, 2],
    [0, 3],
    [0, 4],
    [1, 4],
    [2, 4],
    [2, 5],
    [3, 5],
    [4, 5],
    [5, 5],
    [5, 6],
    [5, 7],
    [6, 7],
    [7, 7],
    [8, 7],
    [9, 7],
    [9, 8],
    [9, 9],
    [10, 9],
    [10, 10],
    [11, 10],
    [11, 11],
    [12, 11],
    [12, 12],
    [12, 13],
    [12, 14],
    [12, 15],
    [13, 15],
    [13, 16],
    [14, 16],
    [15, 16],
    [16, 16],
    [17, 16],
    [18, 16],
    [19, 16],
    [19, 17],
    [19, 18],
    [19, 19],
    [20, 19],
    [20, 20],
    [20, 21],
    [21, 21],
    [21, 22],
    [21, 23],
    [22, 23],
    [23, 23],
    [23, 24],
    [24, 24],
    [25, 24],
    [26, 24]],
   u'server': u'left'},
  {u'left': u'team_b',
   u'left_score': 16,
   u'left_timeouts': [[4, 8], [12, 19]],
   u'right': u'team_a',
   u'right_score': 25,

In [32]:
# This will return two lists, one of scores in format [a, b], another list of [1 if timeout taken, 0 otherwise]
def get_naive_timeout_datapoints(matches):
    X = []
    y = []
    for match in matches:
        for game in match['games']:
            timeouts = []
            timeouts.extend(
                [tuple(timeout) for timeout in game.get('left_timeouts', [])
            ])
            timeouts.extend([
                tuple(timeout) for timeout in game.get('right_timeouts', [])
            ])
            timeouts.extend([
                tuple(timeout) for timeout in game.get('middle_timeouts', [])
            ])
            for score in game['scores']:
                X.append(score)
                score = tuple(score)
                if score in timeouts:
                    y.append(1)
                else:
                    y.append(0)
    return np.array(X), np.array(y)

In [56]:
def train_logistic_regression(X, y):
    model = sklearn.linear_model.LogisticRegression(verbose=1, solver='lbfgs')
    model.fit(X, y)
    return model, model.score(X, y)

X, y = get_naive_timeout_datapoints(GAMES)
logistic_model, score = train_logistic_regression(X, y)
print "base_score", 1.0 - np.mean(y)
print score

base_score 0.9705318714460746
0.9705318714460746


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [55]:
logistic_model.predict([[0, 10]])

array([0])